# Part 1: Exploratory Data Quality Analysis
Profiles the raw customer data and generates a data quality report.

In [ ]:
import pandas as pd
import re
import os
from datetime import datetime

# -- Load Data --
df = pd.read_csv("customers_raw.csv", dtype=str)
df.columns = df.columns.str.strip()

TOTAL_ROWS = len(df)
print(f"Loaded {TOTAL_ROWS} rows, {len(df.columns)} columns\n")

report_lines = []

def add(text=""):
    report_lines.append(text)
    print(text)

# -- SECTION 1: COMPLETENESS --
add("DATA QUALITY PROFILE REPORT")
add("=" * 60)
add()
add("COMPLETENESS:")
add("-" * 40)

completeness = {}
for col in df.columns:
    missing_mask = df[col].isna() | df[col].str.strip().eq("")
    missing_count = missing_mask.sum()
    pct = round((1 - missing_count / TOTAL_ROWS) * 100)
    completeness[col] = {"missing": int(missing_count), "pct": pct}
    status = "pass" if missing_count == 0 else f"fail ({missing_count} missing)"
    add(f"  - {col}: {pct}%  {status}")

add()

# -- SECTION 2: DATA TYPES --
add("DATA TYPES (detected vs expected):")
add("-" * 40)

expected_types = {
    "customer_id":    ("INT",    "Integer"),
    "first_name":     ("STRING", "String"),
    "last_name":      ("STRING", "String"),
    "email":          ("STRING", "String"),
    "phone":          ("STRING", "String"),
    "date_of_birth":  ("DATE",   "Date (YYYY-MM-DD)"),
    "address":        ("STRING", "String"),
    "income":         ("NUMERIC","Numeric"),
    "account_status": ("STRING", "String"),
    "created_date":   ("DATE",   "Date (YYYY-MM-DD)"),
}

def looks_like_int(series):
    try:
        series.dropna().str.strip().astype(int)
        return True
    except:
        return False

def looks_like_numeric(series):
    try:
        series.dropna().str.strip().astype(float)
        return True
    except:
        return False

def looks_like_date(series):
    valid = 0
    total = 0
    for v in series.dropna():
        v = v.strip()
        if v == "":
            continue
        total += 1
        for fmt in ("%Y-%m-%d", "%Y/%m/%d", "%m/%d/%Y"):
            try:
                datetime.strptime(v, fmt)
                valid += 1
                break
            except:
                pass
    return total > 0 and valid == total

for col, (exp_code, exp_label) in expected_types.items():
    if col not in df.columns:
        add(f"  - {col}: MISSING COLUMN X")
        continue
    series = df[col].dropna()
    if exp_code == "INT":
        actual = "INT OK" if looks_like_int(series) else "STRING FAIL (should be INT)"
    elif exp_code == "NUMERIC":
        actual = "NUMERIC OK" if looks_like_numeric(series) else "STRING FAIL (should be NUMERIC)"
    elif exp_code == "DATE":
        actual = "DATE OK" if looks_like_date(series) else f"STRING FAIL (should be DATE - {exp_label})"
    else:
        actual = "STRING OK"
    add(f"  - {col}: {actual}")

add()

# -- SECTION 3: QUALITY ISSUES --
add("QUALITY ISSUES:")
add("-" * 40)

issues = []

bad_rows_fname = df[df["first_name"].isna() | df["first_name"].str.strip().eq("")]
if not bad_rows_fname.empty:
    issues.append({"severity": "High", "description": "Missing first_name", "example": f"IDs: {bad_rows_fname['customer_id'].tolist()}"})

bad_rows_lname = df[df["last_name"].isna() | df["last_name"].str.strip().eq("")]
if not bad_rows_lname.empty:
    issues.append({"severity": "High", "description": "Missing last_name", "example": f"IDs: {bad_rows_lname['customer_id'].tolist()}"})

bad_rows_status = df[df["account_status"].isna() | df["account_status"].str.strip().eq("")]
if not bad_rows_status.empty:
    issues.append({"severity": "Critical", "description": "Missing account_status", "example": f"IDs: {bad_rows_status['customer_id'].tolist()}"})

VALID_STATUSES = {"active", "inactive", "suspended"}
bad_rows_val_status = df[df["account_status"].notna() & ~df["account_status"].str.strip().str.lower().isin(VALID_STATUSES)]
if not bad_rows_val_status.empty:
    for _, row in bad_rows_val_status.iterrows():
        issues.append({"severity": "Critical", "description": "Invalid account_status", "example": f"ID {row['customer_id']}: {row['account_status']}"})

def parse_date(val):
    if pd.isna(val) or val.strip() == "": return None
    for fmt in ("%Y-%m-%d", "%Y/%m/%d", "%m/%d/%Y"):
        try: return datetime.strptime(val.strip(), fmt)
        except: pass
    return "INVALID"

for col in ["date_of_birth", "created_date"]:
    for _, row in df.iterrows():
        res = parse_date(row[col])
        if res == "INVALID":
            issues.append({"severity": "Critical", "description": f"Invalid date in {col}", "example": f"ID {row['customer_id']}: {row[col]}"})

STANDARD_PHONE = re.compile(r"^\d{3}-\d{3}-\d{4}$")
for _, row in df.iterrows():
    phone = str(row["phone"]).strip() if not pd.isna(row["phone"]) else ""
    if phone and not STANDARD_PHONE.match(phone):
        issues.append({"severity": "Medium", "description": "Non-standard phone format", "example": f"ID {row['customer_id']}: {phone}"})

dupes = df[df.duplicated(subset=["customer_id"], keep=False)]
if not dupes.empty:
    issues.append({"severity": "Critical", "description": "Duplicate customer_ids", "example": f"IDs: {dupes['customer_id'].unique().tolist()}"})

for i, issue in enumerate(issues, 1):
    add(f"{i}. [{issue['severity']}] {issue['description']}")
    add(f"   - {issue['example']}")

add()
add("UNIQUENESS CHECK:")
add("-" * 40)
if TOTAL_ROWS > 0:
    if dupes.empty:
        add(f"  - customer_id: All {TOTAL_ROWS} values are unique OK")
    else:
        add("  - customer_id: DUPLICATES FOUND FAIL")
add()

# -- SECTION 4: SEVERITY SUMMARY --
add("SEVERITY SUMMARY:")
add("-" * 40)
severity_counts = {"Critical": 0, "High": 0, "Medium": 0}
for issue in issues:
    severity_counts[issue["severity"]] += 1

add(f"  - Critical: {severity_counts['Critical']}")
add(f"  - High:     {severity_counts['High']}")
add(f"  - Medium:   {severity_counts['Medium']}")
add()
add("END OF REPORT")
add("=" * 60)

os.makedirs("outputs", exist_ok=True)
with open("outputs/data_quality_report.txt", "w") as f:
    f.write("\n".join(report_lines))

print("\nReport saved to outputs/data_quality_report.txt")